In [1]:
data = """Al => ThF
Al => ThRnFAr
B => BCa
B => TiB
B => TiRnFAr
Ca => CaCa
Ca => PB
Ca => PRnFAr
Ca => SiRnFYFAr
Ca => SiRnMgAr
Ca => SiTh
F => CaF
F => PMg
F => SiAl
H => CRnAlAr
H => CRnFYFYFAr
H => CRnFYMgAr
H => CRnMgYFAr
H => HCa
H => NRnFYFAr
H => NRnMgAr
H => NTh
H => OB
H => ORnFAr
Mg => BF
Mg => TiMg
N => CRnFAr
N => HSi
O => CRnFYFAr
O => CRnMgAr
O => HP
O => NRnFAr
O => OTi
P => CaP
P => PTi
P => SiRnFAr
Si => CaSi
Th => ThCa
Ti => BP
Ti => TiTi
e => HF
e => NAl
e => OMg

CRnCaSiRnBSiRnFArTiBPTiTiBFArPBCaSiThSiRnTiBPBPMgArCaSiRnTiMgArCaSiThCaSiRnFArRnSiRnFArTiTiBFArCaCaSiRnSiThCaCaSiRnMgArFYSiRnFYCaFArSiThCaSiThPBPTiMgArCaPRnSiAlArPBCaCaSiRnFYSiThCaRnFArArCaCaSiRnPBSiRnFArMgYCaCaCaCaSiThCaCaSiAlArCaCaSiRnPBSiAlArBCaCaCaCaSiThCaPBSiThPBPBCaSiRnFYFArSiThCaSiRnFArBCaCaSiRnFYFArSiThCaPBSiThCaSiRnPMgArRnFArPTiBCaPRnFArCaCaCaCaSiRnCaCaSiRnFYFArFArBCaSiThFArThSiThSiRnTiRnPMgArFArCaSiThCaPBCaSiRnBFArCaCaPRnCaCaPMgArSiRnFYFArCaSiThRnPBPMgAr"""

In [ ]:
import re
split = lambda s:re.findall('[A-Z][a-z]?', s)
lines = data.splitlines()
*_reac, _, mol = lines

forward = dict()
backward = dict()

for l in _reac:
    r, comp = l.split(' => ')
    forward.setdefault(r, []).append(split(comp))
    backward[comp] = r
forward,backward


In [61]:
def settify(f):
    def _inner(*args, **kwargs):
        return set(f(*args, **kwargs))
    return _inner

@settify
def iter_all(molecule):
    molecule = list(molecule)
    for i in range(len(molecule)):
        if molecule[i] in forward:
            for rep in forward[molecule[i]]:
                assert isinstance(rep, list)
                yield tuple(molecule[:i]+rep+molecule[i+1:])

If one of these is in a molecule, then this section is terminal


In [176]:
terminal = {atom for k in backward.keys() for atom in split(k)} - set(backward.values())
terminal

{'Ar', 'C', 'Rn', 'Y'}

In [180]:
list(sorted([[t for t in split(k) if t in terminal] for k in backward.keys() if set(split(k)).intersection(terminal)]))

[['C', 'Rn', 'Ar'],
 ['C', 'Rn', 'Ar'],
 ['C', 'Rn', 'Ar'],
 ['C', 'Rn', 'Y', 'Ar'],
 ['C', 'Rn', 'Y', 'Ar'],
 ['C', 'Rn', 'Y', 'Ar'],
 ['C', 'Rn', 'Y', 'Y', 'Ar'],
 ['Rn', 'Ar'],
 ['Rn', 'Ar'],
 ['Rn', 'Ar'],
 ['Rn', 'Ar'],
 ['Rn', 'Ar'],
 ['Rn', 'Ar'],
 ['Rn', 'Ar'],
 ['Rn', 'Ar'],
 ['Rn', 'Y', 'Ar'],
 ['Rn', 'Y', 'Ar']]

In [182]:
[atom for atom in split(mol) if atom in terminal]

['C',
 'Rn',
 'Rn',
 'Rn',
 'Ar',
 'Ar',
 'Rn',
 'Ar',
 'Rn',
 'Ar',
 'Rn',
 'Ar',
 'Rn',
 'Rn',
 'Ar',
 'Ar',
 'Rn',
 'Rn',
 'Ar',
 'Y',
 'Rn',
 'Y',
 'Ar',
 'Ar',
 'Rn',
 'Ar',
 'Rn',
 'Y',
 'Rn',
 'Ar',
 'Ar',
 'Rn',
 'Rn',
 'Ar',
 'Y',
 'Ar',
 'Rn',
 'Ar',
 'Rn',
 'Y',
 'Ar',
 'Rn',
 'Ar',
 'Rn',
 'Y',
 'Ar',
 'Rn',
 'Ar',
 'Rn',
 'Ar',
 'Rn',
 'Ar',
 'Rn',
 'Rn',
 'Y',
 'Ar',
 'Ar',
 'Ar',
 'Rn',
 'Rn',
 'Ar',
 'Ar',
 'Rn',
 'Ar',
 'Rn',
 'Ar',
 'Rn',
 'Y',
 'Ar',
 'Rn',
 'Ar']

In [62]:
len(iter_all(split(mol)))

509

In [284]:
start = ['e']
seen = set(start)
for i in range(2):
    s2 = set()
    for s in start:
        s2 = s2.union(iter_all(s))
    seen.update(s2)
    start = s2


In [285]:
for s in sorted(start):
    print(''.join(s))

CRnAlArF
CRnFArAl
CRnFYFArMg
CRnFYFYFArF
CRnFYMgArF
CRnMgArMg
CRnMgYFArF
HCaF
HPMg
HSiAl
NRnFArMg
NRnFYFArF
NRnMgArF
NThF
NThRnFAr
OBF
ORnFArF
OTiMg


In [40]:
allk = [split(k) for k in backward.keys()]

allstart = {k[0] for k in allk}
allend = {k[-1] for k in allk}
allstart, allend


({'B', 'C', 'Ca', 'H', 'N', 'O', 'P', 'Si', 'Th', 'Ti'},
 {'Al', 'Ar', 'B', 'Ca', 'F', 'Mg', 'P', 'Si', 'Th', 'Ti'})

In [88]:
short_keys = [k for k in allk if len(k) == 2]
long_keys = [k for k in allk if len(k) > 2]
long_keys_2 = [k for k in long_keys if k[1:-1] not in allk]
must_reduce = {k:v for k,v in backward.items() if split(k) in long_keys_2}
must_reduce

{'ThRnFAr': 'Al',
 'TiRnFAr': 'B',
 'PRnFAr': 'Ca',
 'SiRnFYFAr': 'Ca',
 'SiRnMgAr': 'Ca',
 'CRnAlAr': 'H',
 'CRnFYFYFAr': 'H',
 'CRnFYMgAr': 'H',
 'CRnMgYFAr': 'H',
 'NRnFYFAr': 'H',
 'NRnMgAr': 'H',
 'ORnFAr': 'H',
 'CRnFAr': 'N',
 'CRnFYFAr': 'O',
 'CRnMgAr': 'O',
 'NRnFAr': 'O',
 'SiRnFAr': 'P'}

In [90]:
'abcxccc'.replace('c', 'x')

'abxxxxx'

In [86]:
long_keys_2

[['Rn', 'F'],
 ['Rn', 'F'],
 ['Rn', 'F'],
 ['Rn', 'F', 'Y', 'F'],
 ['Rn', 'Mg'],
 ['Rn', 'Al'],
 ['Rn', 'F', 'Y', 'F', 'Y', 'F'],
 ['Rn', 'F', 'Y', 'Mg'],
 ['Rn', 'Mg', 'Y', 'F'],
 ['Rn', 'F', 'Y', 'F'],
 ['Rn', 'Mg'],
 ['Rn', 'F'],
 ['Rn', 'F'],
 ['Rn', 'F', 'Y', 'F'],
 ['Rn', 'Mg'],
 ['Rn', 'F'],
 ['Rn', 'F']]

In [42]:
allend - allstart

{'Al', 'Ar', 'F', 'Mg'}

In [110]:
def force_reduce(mol):
    for mr,mv in must_reduce.items():
        mol = mol.replace(mr,mv)
    return mol

In [ ]:
'abcdef'[::-1].

In [309]:
re.findall('..Rn', mol)

['SiRn',
 'SiRn',
 'SiRn',
 'SiRn',
 'SiRn',
 'ArRn',
 'SiRn',
 'SiRn',
 'SiRn',
 'SiRn',
 'aPRn',
 'SiRn',
 'CaRn',
 'SiRn',
 'SiRn',
 'SiRn',
 'SiRn',
 'SiRn',
 'SiRn',
 'SiRn',
 'ArRn',
 'aPRn',
 'SiRn',
 'SiRn',
 'SiRn',
 'TiRn',
 'SiRn',
 'aPRn',
 'SiRn',
 'ThRn']

In [339]:
@settify
def reduce_ar_molecule(mol:str):
    """
    All Ar reduction are with Ar at the end of the reduction; so we _know_, that no reaction after an Ar must be independant, 
    at least until the reduction does not return a submolecule that does not finish in Ar.
    """
    
    ar_cut = mol.index('Ar')+2
    
        
    
    
    for i in range(ar_cut):
        start, end = mol[:i],mol[i:]
        for k,v in backward.items():
                if end.startswith(k):
                    newm = start + v+ end[len(k):]
                    yield newm


@settify
def _iter_reduce(sub):
    yield sub
    for i in range(len(sub)):
        start, end = sub[:i],sub[i:]
        for k,v in backward.items():
            if end.startswith(k):
                newm = start + v+ end[len(k):]
                yield newm
    




def reduce_next_rn(mol:str):

    prev, rest = mol.split('Rn', maxsplit=1)
    return list(x+'Rn'+rest for x in _iter_reduce(prev))

reduce_next_rn(mol[3:])





['CaSiRnBSiRnFArTiBPTiTiBFArPBCaSiThSiRnTiBPBPMgArCaSiRnTiMgArCaSiThCaSiRnFArRnSiRnFArTiTiBFArCaCaSiRnSiThCaCaSiRnMgArFYSiRnFYCaFArSiThCaSiThPBPTiMgArCaPRnSiAlArPBCaCaSiRnFYSiThCaRnFArArCaCaSiRnPBSiRnFArMgYCaCaCaCaSiThCaCaSiAlArCaCaSiRnPBSiAlArBCaCaCaCaSiThCaPBSiThPBPBCaSiRnFYFArSiThCaSiRnFArBCaCaSiRnFYFArSiThCaPBSiThCaSiRnPMgArRnFArPTiBCaPRnFArCaCaCaCaSiRnCaCaSiRnFYFArFArBCaSiThFArThSiThSiRnTiRnPMgArFArCaSiThCaPBCaSiRnBFArCaCaPRnCaCaPMgArSiRnFYFArCaSiThRnPBPMgAr',
 'SiRnBSiRnFArTiBPTiTiBFArPBCaSiThSiRnTiBPBPMgArCaSiRnTiMgArCaSiThCaSiRnFArRnSiRnFArTiTiBFArCaCaSiRnSiThCaCaSiRnMgArFYSiRnFYCaFArSiThCaSiThPBPTiMgArCaPRnSiAlArPBCaCaSiRnFYSiThCaRnFArArCaCaSiRnPBSiRnFArMgYCaCaCaCaSiThCaCaSiAlArCaCaSiRnPBSiAlArBCaCaCaCaSiThCaPBSiThPBPBCaSiRnFYFArSiThCaSiRnFArBCaCaSiRnFYFArSiThCaPBSiThCaSiRnPMgArRnFArPTiBCaPRnFArCaCaCaCaSiRnCaCaSiRnFYFArFArBCaSiThFArThSiThSiRnTiRnPMgArFArCaSiThCaPBCaSiRnBFArCaCaPRnCaCaPMgArSiRnFYFArCaSiThRnPBPMgAr']

In [434]:
lmol = split(mol)

In [450]:
lmol[1:5]

['Rn', 'Ca', 'Si', 'Rn']

In [452]:
ll = list([(i,l)  for (i,l) in enumerate(lmol) if l in ['Ar','Rn']])
for (ia,a),(ib,b),(ic,c) in zip(ll[:-2], ll[1:-1],ll[2:]):
    if (a,b,c) == ('Rn','Rn', 'Ar'):
        print(ia,ib,ic, lmol[ia:ic+1])


4 7 9 ['Rn', 'B', 'Si', 'Rn', 'F', 'Ar']
46 48 50 ['Rn', 'Si', 'Rn', 'F', 'Ar']
59 65 67 ['Rn', 'Si', 'Th', 'Ca', 'Ca', 'Si', 'Rn', 'Mg', 'Ar']
99 105 107 ['Rn', 'F', 'Y', 'Si', 'Th', 'Ca', 'Rn', 'F', 'Ar']
112 116 118 ['Rn', 'P', 'B', 'Si', 'Rn', 'F', 'Ar']
209 213 217 ['Rn', 'Ca', 'Ca', 'Si', 'Rn', 'F', 'Y', 'F', 'Ar']
230 232 235 ['Rn', 'Ti', 'Rn', 'P', 'Mg', 'Ar']


In [432]:
rnar = list(re.finditer('Rn|Ar', mol))


def _iter_reduce_to_one(sub):
    yield sub
    for i in range(len(sub)):
        start, end = sub[:i],sub[i:]
        for k,v in backward.items():
            if end.startswith(k):
                newm = start + v+ end[len(k):]
                yield from _iter_reduce_to_one(newm)

seq = []
for a,b in zip(rnar[:-1], rnar[1:]):
    seq.append(a.group())
    
    if a.group() == b.group() == 'Rn':
        span = mol[a.span()[1]:b.span()[0]]
        
        seq.append(set(_iter_reduce_to_one(span)))
    if a.group() == 'Rn' and b.group() == 'Ar':
        span = mol[a.span()[1]:b.span()[0]]
        #print('Rn|'+ span+'Ar')
        seq.append((set(_iter_reduce_to_one(span))))
    if a.group() == 'Ar' and b.group() == 'Ar':
        span = mol[a.span()[1]:b.span()[0]]
        if span:
            seq.append(span)

seq.append(b)
seq



for i in range(len(seq)-4):
    a,b,c,d = seq[i],seq[i+1], seq[i+2], seq[i+3]
    #if b == 'Rn' and d == 'Ar' and (a != 'Ar') and len(c) == 1:
    #print(a,b,c,d)
seq    

['Rn',
 {'CaSi', 'Si'},
 'Rn',
 {'BSi'},
 'Rn',
 {'F'},
 'Ar',
 'TiBPTiTiBF',
 'Ar',
 'Rn',
 {'BCaF',
  'BCaPMg',
  'BF',
  'BPBF',
  'BPBPMg',
  'BPMg',
  'BPTiMg',
  'Mg',
  'TiBCaF',
  'TiBCaPMg',
  'TiBF',
  'TiBPBF',
  'TiBPBPMg',
  'TiBPMg',
  'TiBPTiMg',
  'TiMg',
  'TiTiBF',
  'TiTiBPMg',
  'TiTiMg',
  'TiTiTiMg'},
 'Ar',
 'Rn',
 {'Mg', 'TiMg'},
 'Ar',
 'Rn',
 {'F'},
 'Ar',
 'Rn',
 {'Si'},
 'Rn',
 {'F'},
 'Ar',
 'TiTiBF',
 'Ar',
 'Rn',
 {'CaCaCaSi', 'CaCaSi', 'CaSi', 'Si', 'SiThCaCaSi', 'SiThCaSi', 'SiThSi'},
 'Rn',
 {'Mg'},
 'Ar',
 'Rn',
 {'FYCaF', 'FYF'},
 'Ar',
 'SiThCaSiThPBPTiMg',
 'Ar',
 'Rn',
 {'F', 'SiAl'},
 'Ar',
 'Rn',
 {'FYCa', 'FYCaCa', 'FYSiTh', 'FYSiThCa'},
 'Rn',
 {'F'},
 'Ar',
 'Ar',
 'Rn',
 {'CaSi', 'PBSi', 'Si'},
 'Rn',
 {'F'},
 'Ar',
 'MgYCaCaCaCaSiThCaCaSiAl',
 'Ar',
 'Rn',
 {'CaF', 'CaSiAl', 'F', 'PBF', 'PBSiAl', 'PMg', 'SiAl'},
 'Ar',
 'Rn',
 {'FYF'},
 'Ar',
 'Rn',
 {'F'},
 'Ar',
 'Rn',
 {'FYF'},
 'Ar',
 'Rn',
 {'F', 'PMg'},
 'Ar',
 'Rn',
 {'F'},
 'Ar',
 '

In [351]:
    

@settify
def reduce_molecule_set(mols):
    for m in mols:
        if split(m[:10])[1] in 'Rn':
            yield from reduce_ar_molecule(m)
        else:
            yield from m[:3]+reduce_next_rn(m)
mset = {mol}
seen = set()
for it in range(35):
    mset = reduce_molecule_set(mset)
    intersect = seen.intersection(mset)
    sarl = [m.index('Ar')+2 for m in mset]
    seen.update(mset)
    print(f" -- {it+1: 4}:{len(mset): 3} molecules {min(len(x) for x in mset), max(len(x) for x in mset)} , Min Ar at {min(sarl)}  int:{len(intersect)}--")

 --    1:  2 molecules (462, 466) , Min Ar at 15  int:0--
 --    2:  6 molecules (460, 462) , Min Ar at 21  int:0--
 --    3: 16 molecules (458, 462) , Min Ar at 19  int:0--
 --    4: 25 molecules (456, 462) , Min Ar at 17  int:0--
 --    5: 26 molecules (454, 460) , Min Ar at 15  int:0--
 --    6: 20 molecules (454, 458) , Min Ar at 15  int:0--
 --    7: 12 molecules (452, 456) , Min Ar at 13  int:0--
 --    8:  6 molecules (452, 454) , Min Ar at 13  int:0--
 --    9:  3 molecules (450, 452) , Min Ar at 11  int:0--
 --   10:  2 molecules (446, 450) , Min Ar at 11  int:0--
 --   11:  9 molecules (444, 446) , Min Ar at 27  int:0--
 --   12: 33 molecules (442, 446) , Min Ar at 25  int:0--
 --   13: 68 molecules (440, 446) , Min Ar at 23  int:0--
 --   14: 94 molecules (438, 444) , Min Ar at 21  int:0--
 --   15: 98 molecules (436, 442) , Min Ar at 19  int:0--
 --   16: 85 molecules (434, 440) , Min Ar at 17  int:0--
 --   17: 67 molecules (432, 438) , Min Ar at 15  int:0--
 --   18: 54 m

In [381]:
# for m in sorted(mset):
#    ar_cut = m.index('Ar')+2
   
#    print(m[:ar_cut])

In [382]:
atom_index  = {atom:set() for k in backward.keys() for atom in split(k)}
atom_index

{'Th': set(),
 'F': set(),
 'Rn': set(),
 'Ar': set(),
 'B': set(),
 'Ca': set(),
 'Ti': set(),
 'P': set(),
 'Si': set(),
 'Y': set(),
 'Mg': set(),
 'Al': set(),
 'C': set(),
 'H': set(),
 'N': set(),
 'O': set()}

In [383]:
for k in backward.keys():
    key = split(k)
    for at in key:
        atom_index[at].add(key.index(at))
atom_index

{'Th': {0, 1},
 'F': {1, 2, 4},
 'Rn': {1},
 'Ar': {3, 5, 7},
 'B': {0, 1},
 'Ca': {0, 1},
 'Ti': {0, 1},
 'P': {0, 1},
 'Si': {0, 1},
 'Y': {3},
 'Mg': {1, 2, 4},
 'Al': {1, 2},
 'C': {0},
 'H': {0},
 'N': {0},
 'O': {0}}

In [408]:
for k,v in backward.items():
    if "Rn" not in k:
        continue
    print(f"{k:<10}, {v:>7}")

ThRnFAr   ,      Al
TiRnFAr   ,       B
PRnFAr    ,      Ca
SiRnFYFAr ,      Ca
SiRnMgAr  ,      Ca
CRnAlAr   ,       H
CRnFYFYFAr,       H
CRnFYMgAr ,       H
CRnMgYFAr ,       H
NRnFYFAr  ,       H
NRnMgAr   ,       H
ORnFAr    ,       H
CRnFAr    ,       N
CRnFYFAr  ,       O
CRnMgAr   ,       O
NRnFAr    ,       O
SiRnFAr   ,       P


In [384]:
for k,v in backward.items():
    print(f"{k:<10}, {v:>7}")

ThF       ,      Al
ThRnFAr   ,      Al
BCa       ,       B
TiB       ,       B
TiRnFAr   ,       B
CaCa      ,      Ca
PB        ,      Ca
PRnFAr    ,      Ca
SiRnFYFAr ,      Ca
SiRnMgAr  ,      Ca
SiTh      ,      Ca
CaF       ,       F
PMg       ,       F
SiAl      ,       F
CRnAlAr   ,       H
CRnFYFYFAr,       H
CRnFYMgAr ,       H
CRnMgYFAr ,       H
HCa       ,       H
NRnFYFAr  ,       H
NRnMgAr   ,       H
NTh       ,       H
OB        ,       H
ORnFAr    ,       H
BF        ,      Mg
TiMg      ,      Mg
CRnFAr    ,       N
HSi       ,       N
CRnFYFAr  ,       O
CRnMgAr   ,       O
HP        ,       O
NRnFAr    ,       O
OTi       ,       O
CaP       ,       P
PTi       ,       P
SiRnFAr   ,       P
CaSi      ,      Si
ThCa      ,      Th
BP        ,      Ti
TiTi      ,      Ti
HF        ,       e
NAl       ,       e
OMg       ,       e


In [211]:
# @settify
# def red(mols):
#     for i in range(len(mol)):
#             start, end = mol[:i],mol[i:]
            
#             for k,v in backward.items():
#                 if end.startswith(k):
#                     newm = start + v+ end[len(k):]
#                     yield force_reduce(newm)


# mols = [mol]
# # mols = ['CRnSiRnTiBSiRnSiRnTiTiBFArFYPMgArSiRnCaFArRnPTiMgArSiRnSiAlYCaCaSiThPBPRnSiAlArPMgArCaSiThCaSiRnFArBPTiTiRnCaSiAlArCaPBCaFArSiRnTiMgArPMgYMgArSiRnSiRnMgArPMgArTiMg']
# seen = set()
# for it in range(250):
#     lens = [len(x) for x in mols]
#     
#     print(f"{it}:{len(mols)} molecules {min(lens), max(lens)}")
#     mols = red(list(mols)[:10])
#     #print(f'{len(mols)} generated')
#     mols = mols - seen
#     #print(f'{len(mols)} non duplicates')
#     seen.update(mols)
#     if 'e' in mols:
#         print('DONE in it:', it+1)
#         break

0:1 molecules (468, 468), Min Ar at 25
1:86 molecules (391, 400), Min Ar at 25


ValueError: min() iterable argument is empty

In [115]:
# 164 is way too low
n = next(iter(mols))

In [157]:
ms = [mol]
while True:        
    if 'Ar' not in ms[-1]:
        break
    ind = ms[-1].index('Ar')
    mx = ms[-1]
    ms = ms[:-1] + [mx[:ind+2], mx[ind+2:]]

ms

['CRnCaSiRnBSiRnFAr',
 'TiBPTiTiBFAr',
 'PBCaSiThSiRnTiBPBPMgAr',
 'CaSiRnTiMgAr',
 'CaSiThCaSiRnFAr',
 'RnSiRnFAr',
 'TiTiBFAr',
 'CaCaSiRnSiThCaCaSiRnMgAr',
 'FYSiRnFYCaFAr',
 'SiThCaSiThPBPTiMgAr',
 'CaPRnSiAlAr',
 'PBCaCaSiRnFYSiThCaRnFAr',
 'Ar',
 'CaCaSiRnPBSiRnFAr',
 'MgYCaCaCaCaSiThCaCaSiAlAr',
 'CaCaSiRnPBSiAlAr',
 'BCaCaCaCaSiThCaPBSiThPBPBCaSiRnFYFAr',
 'SiThCaSiRnFAr',
 'BCaCaSiRnFYFAr',
 'SiThCaPBSiThCaSiRnPMgAr',
 'RnFAr',
 'PTiBCaPRnFAr',
 'CaCaCaCaSiRnCaCaSiRnFYFAr',
 'FAr',
 'BCaSiThFAr',
 'ThSiThSiRnTiRnPMgAr',
 'FAr',
 'CaSiThCaPBCaSiRnBFAr',
 'CaCaPRnCaCaPMgAr',
 'SiRnFYFAr',
 'CaSiThRnPBPMgAr',
 '']

In [407]:
backward

{'ThF': 'Al',
 'ThRnFAr': 'Al',
 'BCa': 'B',
 'TiB': 'B',
 'TiRnFAr': 'B',
 'CaCa': 'Ca',
 'PB': 'Ca',
 'PRnFAr': 'Ca',
 'SiRnFYFAr': 'Ca',
 'SiRnMgAr': 'Ca',
 'SiTh': 'Ca',
 'CaF': 'F',
 'PMg': 'F',
 'SiAl': 'F',
 'CRnAlAr': 'H',
 'CRnFYFYFAr': 'H',
 'CRnFYMgAr': 'H',
 'CRnMgYFAr': 'H',
 'HCa': 'H',
 'NRnFYFAr': 'H',
 'NRnMgAr': 'H',
 'NTh': 'H',
 'OB': 'H',
 'ORnFAr': 'H',
 'BF': 'Mg',
 'TiMg': 'Mg',
 'CRnFAr': 'N',
 'HSi': 'N',
 'CRnFYFAr': 'O',
 'CRnMgAr': 'O',
 'HP': 'O',
 'NRnFAr': 'O',
 'OTi': 'O',
 'CaP': 'P',
 'PTi': 'P',
 'SiRnFAr': 'P',
 'CaSi': 'Si',
 'ThCa': 'Th',
 'BP': 'Ti',
 'TiTi': 'Ti',
 'HF': 'e',
 'NAl': 'e',
 'OMg': 'e'}

In [186]:
iter_all('NRnFYFAr')

{('C', 'Rn', 'F', 'Ar', 'R', 'n', 'F', 'Y', 'F', 'A', 'r'),
 ('H', 'Si', 'R', 'n', 'F', 'Y', 'F', 'A', 'r'),
 ('N', 'R', 'n', 'Ca', 'F', 'Y', 'F', 'A', 'r'),
 ('N', 'R', 'n', 'F', 'Y', 'Ca', 'F', 'A', 'r'),
 ('N', 'R', 'n', 'F', 'Y', 'P', 'Mg', 'A', 'r'),
 ('N', 'R', 'n', 'F', 'Y', 'Si', 'Al', 'A', 'r'),
 ('N', 'R', 'n', 'P', 'Mg', 'Y', 'F', 'A', 'r'),
 ('N', 'R', 'n', 'Si', 'Al', 'Y', 'F', 'A', 'r')}